In [109]:
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure
import pandas as pd

STRUCTURE_DIR = '/Users/AlexDo/PycharmProjects/VASP_tools'

f = Poscar.from_file(STRUCTURE_DIR+'/'+'Al2O3_POSCAR.POSCAR.vasp')
elements = f.site_symbols
structure = f.structure # This returns the data in the imported file as a pymatgen structure object


d = {'Material': 'Al2O3', 'Structure': [structure]} # I think we want the entire structure object in a single cell of the dataframe
df = pd.DataFrame(d)


In [110]:
import multiprocessing
import os
import logging

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures as SklearnPolynomialFeatures

# locate path to directory containing AtomicNumber.table, AtomicRadii.table AtomicVolume.table, etc
# (needs to do it the hard way becuase python -m sets cwd to wherever python is ran from)

# is this how you do it? Any other imports needed for this module then?
import inspect # used to get a dictionary of classes in a module
from matminer.featurizers import structure


import mastml
from mastml import utils
log = logging.getLogger('mastml')
print('mastml dir: ', mastml.__path__)
MAGPIE_DATA_PATH = os.path.join(mastml.__path__[0], '../magpie/')


class Matminer(BaseEstimator, TransformerMixin):
    def __init__(self, structural_features, structure_col): # _instantiate only needs this
        # assuming dataframe is coming in with a column 'Structure' with coords.
        # where do I need to raise errors
        structural_features = structural_features.split(', ') # structural feature is now cast as a list
        self.structural_features = structural_features # structural feature field of class
        self.structure_col = structure_col

    def fit(self, df, y=None):
        return self

    def transform(self, df, y=None):
        # any possible errors i need to raise?
        for struc_feat in range(len(self.structural_features)): # nested for loop to iterate through structural features list
            for feature_name in inspect.getmembers(structure, inspect.isclass):
              # will get list of all classes in structure module
                if feature_name[0] == self.structural_features[struc_feat]:
                    sf = getattr(structure, self.structural_features[struc_feat])() # instantiates the structure featurizer
                    df = sf.fit_featurize_dataframe(df, self.structure_col)  # fit_featurize_dataframe() works for all
                     # if for loop passed, structural_feature was not a valid structural feature
                    break
                    
        return df  # I think it should return the updated dataframe to concatenate



mastml dir:  ['/Users/AlexDo/PycharmProjects/MAST-ML/mastml']


In [111]:
y = None
Matminer("Dimensionality, CoulombMatrix, SineCoulombMatrix, GlobalSymmetryFeatures", "Structure").fit_transform(df, y)

/Users/AlexDo/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:184: DeprecationWarning:

get_dimensionality is deprecated
find_dimension has been moved topymatgen.analysis.dimensionality.get_dimensionality_gorai this method will be removed in pymatgen v2019.1.1.



,Material,Structure,dimensionality,coulomb matrix,sine coulomb matrix,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric
0,Al2O3,"[[0. 0. 1.93994619] Al, [ 0. ...",3,"[[235.74041833262768, 23.049509471319066, 33.3...","[[235.74041833262768, 10.621430816382633, 19.0...",167,trigonal,3,True


In [78]:
s1 = "Dimensionality"
s2 = "Dimensionality"
s1.__class__.__name__


'str'

('Dimensionality, CoulombMatrix, SineCoulomb, GlobalSymmetryFeatures',
 ['Structure'])